## InfoGCN++ training pipeline
This notebook shows how to prepare data and train the SODE model on COCO-format skeleton `.npy` clips.

In [1]:
from pathlib import Path

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from act_rec.datasets import SkeletonNpyDataset
from act_rec.model.losses import LabelSmoothingCrossEntropy
from act_rec.model.sode import SODE
from act_rec.training import TrainConfig, evaluate, train_one_epoch

In [2]:
# Paths
data_root = Path("../data/")  # folder containing the .npy clips
csv_path = data_root / "labels.csv"

df = pd.read_csv(csv_path)
df = df.dropna()
label_to_idx = {label: idx for idx, label in enumerate(sorted(df["label"].unique()))}
df["label_idx"] = df["label"].map(label_to_idx)
df["skeleton_path"] = df["skeleton_path"].apply(lambda p: str((data_root / p).resolve()))

In [3]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label_idx"],
    random_state=42,
)

In [4]:
window_size = 64
train_dataset = SkeletonNpyDataset(
    train_df["skeleton_path"].tolist(),
    labels=train_df["label_idx"].tolist(),
    window_size=window_size,
    p_interval=(1.0,),
    random_rotation=False,
    preload=True,
)
val_dataset = SkeletonNpyDataset(
    val_df["skeleton_path"].tolist(),
    labels=val_df["label_idx"].tolist(),
    window_size=window_size,
    p_interval=(1.0,),
    random_rotation=False,
    preload=True,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

In [5]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SODE(
    num_class=len(label_to_idx),
    num_point=17,
    num_person=1,
    graph="act_rec.graph.coco.Graph",
    in_channels=3,
    T=window_size,
    n_step=3,
    num_cls=4,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
config = TrainConfig(
    device=device,
    cls_loss=LabelSmoothingCrossEntropy(smoothing=0.1),
    lambda_cls=1.0,
    lambda_recon=0.1,
)


In [6]:
num_epochs = 10
best_val = float("inf")
history = []

for epoch in range(num_epochs):
    train_metrics = train_one_epoch(model, train_loader, optimizer, config)
    val_metrics = evaluate(model, val_loader, config)
    history.append({**train_metrics, **val_metrics})

    print(
        "Epoch {}/{} | train_cls={:.4f} | val_cls={:.4f} | val_top1={:.3f} | val_top5={:.3f}".format(
            epoch + 1,
            num_epochs,
            train_metrics["train_cls_loss"],
            val_metrics["val_cls_loss"],
            val_metrics["val_top1"],
            val_metrics["val_top5"],
        )
    )

    if val_metrics["val_cls_loss"] < best_val:
        best_val = val_metrics["val_cls_loss"]
        torch.save({"model": model.state_dict(), "label_to_idx": label_to_idx}, "sode_best.pt")


/Users/nikita/anaconda3/envs/act-rec/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/10 | train_cls=2.5440 | val_cls=2.4689 | val_top1=0.224 | val_top5=0.662
Epoch 2/10 | train_cls=2.4066 | val_cls=2.4253 | val_top1=0.231 | val_top5=0.651
Epoch 3/10 | train_cls=2.3682 | val_cls=2.3712 | val_top1=0.247 | val_top5=0.737
Epoch 4/10 | train_cls=2.3305 | val_cls=2.3569 | val_top1=0.229 | val_top5=0.707
Epoch 5/10 | train_cls=2.2969 | val_cls=2.3105 | val_top1=0.259 | val_top5=0.789
Epoch 6/10 | train_cls=2.2665 | val_cls=2.3065 | val_top1=0.261 | val_top5=0.773
Epoch 7/10 | train_cls=2.2201 | val_cls=2.2864 | val_top1=0.345 | val_top5=0.780
Epoch 8/10 | train_cls=2.1947 | val_cls=2.2318 | val_top1=0.338 | val_top5=0.825
Epoch 9/10 | train_cls=2.1410 | val_cls=2.1851 | val_top1=0.322 | val_top5=0.828
Epoch 10/10 | train_cls=2.1221 | val_cls=2.1309 | val_top1=0.376 | val_top5=0.834
